In [0]:
-- Create the first table to extract structured information from reviews
CREATE LIVE TABLE reviews_structured AS
SELECT
  ai_query(
    'databricks-meta-llama-3-3-70b-instruct',
    CONCAT(
      'Extract the following information from the review: ',
      'Location (city name, street name, whatever may be mentioned in the review), Service Score (1-5), Product Score (1-5), Product Name (if mentioned) (comma separated if multiple), Atmosphere Score [cleanliness, accessibility, location, etc] (1-5), Urgency (high, low) [set to high if the review is particularly actionable and due to exceptionally good or poor service]. ',
      'If the review doesnt contain an element, leave it blank or set it to zero. For instance, if the review does not mention service, then set service_score = 0. Urgency should always have a value. All scores should be 1-5 (if they are not null), with 1 being the worst and 5 being the best.',
      'Review: ', review
    ),
    responseFormat => '{
                        "type": "json_schema",
                        "json_schema":
                          {
                           "name": "review_extraction",
                           "schema":
                             {
                              "type": "object",
                              "properties":
                               {
                                "location": { "type": "string" },
                                "service_score": { "type": "integer" },
                                "product_score": { "type": "integer" },
                                "product_name": { "type": "string" },
                                "atmosphere_score": { "type": "integer" },
                                "urgency": { "type": "string" }
                               }
                             },
                            "strict": true
                          }
                      }'
  ) AS structured_review, *
FROM retail_prod.media.reviews;


In [0]:
-- Create the second table to parse and transform the structured review data
CREATE LIVE TABLE reviews_structured_gold AS
WITH parsed_reviews AS (
  SELECT 
    parse_json(structured_review):location::string AS location,
    parse_json(structured_review):service_score::int AS service_score,
    parse_json(structured_review):product_score::int AS product_score,
    parse_json(structured_review):atmosphere_score::int AS atmosphere_score,
    parse_json(structured_review):urgency::string AS urgency,
    parse_json(structured_review):product_name::string AS product_name,
    *
  FROM LIVE.reviews_structured
)
SELECT 
  CASE 
    WHEN location = '' THEN NULL 
    ELSE location 
  END AS location,
  NULLIF(service_score, 0) AS service_score,
  NULLIF(product_score, 0) AS product_score,
  NULLIF(atmosphere_score, 0) AS atmosphere_score,
  urgency,
  TRANSFORM(
    FILTER(
      SPLIT(
        TRIM(
          REGEXP_REPLACE(
            product_name, 
            '(?i)\\bcookies?\\b', 
            ''
          )
        ), 
        ','
      ), 
      x -> TRIM(x) != ''
    ), 
    x -> TRIM(x)
  ) AS product_name, 
  review, 
  string(franchiseID) franchiseID, 
  review_date
FROM parsed_reviews;